In [1]:
%pip install wildlife-datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 1.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install wildlife-tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.8/427.8 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 6.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install timm

Note: you may need to restart the kernel to use updated packages.


In [4]:
# from wildlife_tools.features import  SuperPointFeatures
# import wildlife_tools.features
# help(wildlife_tools.features)

In [5]:
# import numpy as np
# from PIL import Image
# import torchvision.transforms as T
# from wildlife_datasets import datasets
# from wildlife_tools.data import WildlifeDataset
# from wildlife_tools.features import SIFTFeatures
# import cv2  # Import OpenCV for SIFT



# # Download dataset (if not already downloaded)
# datasets.CZoo.get_data('../data/CZoo')

# # Load dataset metadata
# metadata_CZoo = datasets.CZoo('../data/CZoo')

# # Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
# transform = T.Compose([
#     T.Resize([224, 224]),  # Resize the image
#     # T.ToPILImage(),  # Convert tensor to PIL Image
#     T.Grayscale(),  # Convert to grayscale
#     # T.ToTensor(),  # Convert PIL Image to numpy array
#     # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
# ])

# # Create datasets with transformations
# dataset = WildlifeDataset(metadata_CZoo.df, metadata_CZoo.root, transform=transform)
# dataset_database_CZoo = WildlifeDataset(metadata_CZoo.df.iloc[100:], metadata_CZoo.root, transform=transform)
# dataset_query_CZoo = WildlifeDataset(metadata_CZoo.df.iloc[:100], metadata_CZoo.root, transform=transform)

# # Initialize SIFT extractor
# sift = cv2.SIFT_create()
# extractor_CZoo =  SuperPointFeatures()

# query_CZoo, database_CZoo = extractor_CZoo(dataset_query_CZoo), extractor_CZoo(dataset_database_CZoo)

# print(f'First 5 query features shape: {[i.shape for i in query_CZoo[:5]]}')
# print(f'First 5 database features shape: {[i.shape for i in database_CZoo[:5]]}')

# import timm
# import pandas as pd
# import torchvision.transforms as T

# from wildlife_tools.data import WildlifeDataset, SplitMetadata
# from wildlife_tools.features import SIFTFeatures
# from wildlife_tools.similarity import MatchDescriptors
# from wildlife_tools.inference import KnnClassifier

# similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.8])
# sim = similarity(query_CZoo, database_CZoo)[0.8]

# print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


# classifier_CZoo = KnnClassifier(k=1, database_labels=dataset_database_CZoo.labels_string)
# predictions_CZoo = classifier_CZoo(sim)

# print("Predictions for 100 test Images:-\n",predictions_CZoo)
# accuracy_CZoo = np.mean(dataset_query_CZoo.labels_string == predictions_CZoo)
# print("Accuracy on CZoo data: {:.2f}%".format(accuracy_CZoo * 100))



# # Nearest neigbour classifier using the similarity
# classifier = KnnClassifier(k=1, database_labels=dataset_database_CZoo.labels_string)
# preds = classifier(sim)
# print("Prediction \t", preds)
# print("Ground truth \t", dataset_query_CZoo.labels_string)

# acc = sum(preds == dataset_query_CZoo.labels_string) / len(dataset_query_CZoo.labels_string)
# print('\n Accuracy: ', acc*100,"%")

